In [69]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re
import sys
sys.path.append('../../../')
import keys as k
from time import sleep
from datetime import datetime, date
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor
import os
import glob

### **Prueba de Extracción Exitosa - App de extracción Tweets (v. 1.0.0)**

In [50]:
tweets_list = []
e = 0
max_result = 20000
dates = pd.DataFrame()
dates["start_date"] = pd.date_range(start='2022-11-01', end='2023-05-31')
dates["end_date"] = dates["start_date"] + pd.DateOffset(days=1)
dates = dates[dates.start_date > datetime(2022, 11, 1)]
dates = dates.astype("str")

In [51]:
dates.head()

,start_date,end_date
1,2022-11-02,2022-11-03
2,2022-11-03,2022-11-04
3,2022-11-04,2022-11-05
4,2022-11-05,2022-11-06
5,2022-11-06,2022-11-07


In [ ]:
for index, row in dates.iterrows():
    tweets_list = []
    naming = row['start_date']
    term = '(bitcoin OR btc OR $BTC OR BTC OR BITCOIN OR $btc OR #btc OR #bitcoin) since:'+row['start_date']+' until:'+row['end_date']+' lang:en'
    print(naming + " in progress!")
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f'{term}').get_items()):  # declare a username
        e += 1
        print(f"Current scrape index: {e}", end="\r", flush=True)
        if i>max_result: #number of tweets you want to scrape
            break
            # print(movie_name, i, tweet)
        try:
            tweets_list.append([tweet.id, tweet.date, tweet.rawContent, tweet.user.id, tweet.user.username, tweet.user.displayname, tweet.user.created, tweet.user.followersCount, tweet.user.verified, tweet.user.location,  tweet.likeCount, tweet.retweetCount, tweet.bookmarkCount, tweet.quoteCount, tweet.pinned, tweet.hashtags, tweet.cashtags,  tweet.mentionedUsers, tweet.lang, tweet.coordinates, tweet.quotedTweet, tweet.retweetedTweet]) 
        except:
            continue
        
        
    tweets_df = pd.DataFrame(tweets_list, columns=['tweet_id', 'datetime', 'text', 'user_id', 'screen_name', 'full_name', 'user_created', 'followers_count', 'user_verified', 'user_location', 'likes', 'retweet_count', 'bookmark_count', 'quote_count', 'pinned', 'hashtags', 'cashtags', 'mentions', 'language', 'coordinates', 'quoted_tweet_id', 'retweeted_tweet_id'])

In [ ]:
tweets_df.head(5)

### **Versión Alfa - App de extracción Tweets (v. 1.0.2)**

In [129]:
class TweetScraper:
    def __init__(self, start_date, end_date, max_result, num_threads):
        self.start_date = start_date
        self.end_date = end_date
        self.max_result = max_result
        self.num_threads = num_threads
        self.tweets_list = []

    def clean_tweet(self, tweet):
        tweet = tweet.lower()
        tweet = re.sub(r'@[^\s]+', '', tweet)
        tweet = re.sub(r'(?<!\w)#(?!bitcoin|btc\b)\w+(?:,)?', '', tweet)
        tweet = re.sub(r'(?<!\w)\$(?!bitcoin|btc\b)\w+(?:,)?', '', tweet)
        tweet = re.sub(r'http[^\s,]+', '', tweet)
        tweet = re.sub(r'[^\w\s,$#]', '', tweet)
        tweet = re.sub(r'\s+', ' ', tweet)
        tweet = re.sub(r'^\s+|\s+$', '', tweet)

        return tweet

        
    def scrape_tweets(self, start, end):
        sub_tweets_list = []
        tweets_ids_csv = pd.read_csv('./dataset/tweets_ids.csv')
        df_tweets_ids = pd.DataFrame(tweets_ids_csv)
        list_tweets_ids = list(df_tweets_ids['tweet_id'])  # Obtener la lista de IDs existentes
        term = '(bitcoin OR btc OR $BTC OR BTC OR BITCOIN OR $btc OR #btc OR #bitcoin) since:{} until:{} lang:en'.format(start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d'))
        print("{} in progress!".format(start.strftime('%Y-%m-%d')))
        
        existing_tweet_ids = set(list_tweets_ids)  # Convertir la lista de IDs existentes en un conjunto
        
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(term, maxEmptyPages=200).get_items()):
            if i > self.max_result:
                break
            if tweet.id in existing_tweet_ids:
                continue
            try:
                cleaned_tweet = self.clean_tweet(tweet.rawContent)
                sub_tweets_list.append([tweet.id, tweet.date, cleaned_tweet, tweet.user.id, tweet.user.username, tweet.user.displayname, tweet.user.created, tweet.user.followersCount, tweet.user.verified, tweet.user.location, tweet.likeCount, tweet.retweetCount, tweet.bookmarkCount, tweet.quoteCount, tweet.hashtags, tweet.cashtags, tweet.mentionedUsers, tweet.lang, tweet.coordinates, tweet.quotedTweet])
                existing_tweet_ids.add(tweet.id)  # Agregar el ID del tweet a la lista de IDs existentes
                
            except:
                continue
        tweets_ids = pd.DataFrame({'tweet_id': list(existing_tweet_ids)})
        tweets_ids.to_csv('./dataset/tweets_ids.csv', index=False)
        return sub_tweets_list
    
    def scrape_parallel(self):
        date_range = pd.date_range(start=self.start_date, end=self.end_date, freq='1D')

        files_csv = glob.glob('./dataset/*.csv')
        filename_list_csv = [os.path.basename(file) for file in files_csv]
        
        output_filename_base = f'tweets_{self.start_date.strftime("%Y-%m-%d")}_{self.end_date.strftime("%Y-%m-%d")}'
        output_filename = output_filename_base + '.csv'

        # Verificar si el nombre de la variable existe en la lista de archivos
        if output_filename in filename_list_csv:
            # Extraer el número incremental del archivo existente
            incremental_number = 1
            while f"{output_filename_base}_{incremental_number}.csv" in filename_list_csv:
                incremental_number += 1
            
            # Construir el nuevo nombre de archivo con el número incremental
            new_output_filename = f"{output_filename_base}_{incremental_number}.csv"
        else:
            # El nombre de la variable no existe en la lista de archivos
            new_output_filename = output_filename_base + ".csv"
    
                
        with ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            futures = []
            for i in range(len(date_range) - 1):
                start = date_range[i]
                end = date_range[i+1]
                futures.append(executor.submit(self.scrape_tweets, start, end))
            
            for future in futures:
                sub_tweets_list = future.result()
                self.tweets_list.extend(sub_tweets_list)
        
        tweets2_df = pd.DataFrame(self.tweets_list, columns=['tweet_id', 'datetime', 'text', 'user_id', 'screen_name', 'full_name', 'user_created', 'followers_count', 'user_verified', 'user_location', 'likes', 'retweet_count', 'bookmark_count', 'quote_count', 'hashtags', 'cashtags', 'mentions', 'language', 'coodinates', 'quoted_tweet_id'])
        tweets2_df.to_csv('./dataset/' + new_output_filename, index=False, sep='|')
        print(f"Output file generated: {new_output_filename}")


# Decorador para medir el tiempo de ejecución
def measure_execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = datetime.now()
        func(*args, **kwargs)
        end_time = datetime.now()
        execution_time = end_time - start_time
        print("Execution time: {}".format(execution_time))
    return wrapper


@measure_execution_time
def main():
    #start_date = datetime(2023, 6, 17)
    #end_date = datetime(2023, 6, 19)
    start_date = datetime.now()
    end_date = start_date + timedelta(hours=48)
    max_result = 2000
    num_threads = 10

    scraper = TweetScraper(start_date, end_date, max_result, num_threads)
    scraper.scrape_parallel()

        # Pausa de 1 hora antes de la próxima ejecución
        #time.sleep(3600)  # 3600 segundos = 1 hora

if __name__ == "__main__":
    main()

2023-06-18 in progress!
2023-06-19 in progress!


Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-2815ae6580246adb'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-28157c95e017662a'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-28154ca6cca8134e'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-281597f42adbafdf'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-2815a1b880cc2696'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-28158fc5eab9f380'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-2815b2751b05ad95'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-281503ed6fc44d31'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-2815d16ca9bec167'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-28158b9b1f398f5a'
Skipping unrecognised entry ID: 'promoted-tweet-1651254815916281864-28159f2c09d89ed5'
Skipping unrecognised entry ID: 'promoted-tweet-165125

Output file generated: tweets_2023-06-18_2023-06-20_6.csv
Execution time: 0:01:13.214905


#### **Leemos los archivos .csv, luego concatenamos, realizamos eliminación de repetidos en la columna `tweet_id` y eliminación de campos vacíos en la columna `text`, creamos un nuevo archivo general .csv**

In [148]:
def clean_data(df_csv):
    df_csv = df_csv.drop_duplicates(subset=['tweet_id', 'text']).dropna(subset=['text'])
    return df_csv

folder_path = './dataset'  # Ruta de la carpeta que contiene los archivos CSV
file_names = [file for file in os.listdir(folder_path) if file.endswith('.csv') and file != 'tweets_ids.csv']

# Lista para almacenar los DataFrames de cada archivo CSV
dataframes = []

concatenated_df = pd.concat([pd.read_csv('./dataset/' + file_name, sep='|') for file_name in file_names], ignore_index=True)

concatenated_df = clean_data(concatenated_df)
# Ver el DataFrame resultante
concatenated_df.to_csv('./dataset/all/tweets_all_concatenated.csv', index=False, sep='|')


In [149]:
data_csv = pd.read_csv('./dataset/all/tweets_all_concatenated.csv', sep='|')
df_csv = pd.DataFrame(data_csv)
df_csv.info()
df_csv.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205445 entries, 0 to 1205444
Data columns (total 20 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   tweet_id         1205445 non-null  int64 
 1   datetime         1205445 non-null  object
 2   text             1205445 non-null  object
 3   user_id          1205445 non-null  int64 
 4   screen_name      1205445 non-null  object
 5   full_name        1205419 non-null  object
 6   user_created     1205445 non-null  object
 7   followers_count  1205445 non-null  int64 
 8   user_verified    1205445 non-null  bool  
 9   user_location    698635 non-null   object
 10  likes            1205445 non-null  int64 
 11  retweet_count    1205445 non-null  int64 
 12  bookmark_count   1205445 non-null  int64 
 13  quote_count      1205445 non-null  int64 
 14  hashtags         561410 non-null   object
 15  cashtags         209218 non-null   object
 16  mentions         529581 non-null   o

,tweet_id,datetime,text,user_id,screen_name,full_name,user_created,followers_count,user_verified,user_location,likes,retweet_count,bookmark_count,quote_count,hashtags,cashtags,mentions,language,coodinates,quoted_tweet_id
0,1642315884097159168,2023-04-01 23:59:54+00:00,price of is an oxymoron,1437471470830096391,BitcoinBombadil,₿itcoin ₿ombadil,2021-09-13 17:41:41+00:00,1103,False,Planet Earth,0,0,0,0,['Bitcoin'],NaN,NaN,en,NaN,https://twitter.com/Psifour/status/16422783395...
1,1642315871220645888,2023-04-01 23:59:51+00:00,updated bitcoin transaction fees bch next bloc...,1118873707785674752,BitcoinFeesCash,Bitcoin Fees,2019-04-18 13:47:34+00:00,1355,False,Outer Space,3,0,0,0,NaN,NaN,NaN,en,NaN,NaN
2,1642315843185827846,2023-04-01 23:59:44+00:00,it is not a black swan if it is the most obvi...,1589245485008683012,iwilldefi,End of Trust,2022-11-06 13:17:15+00:00,125,False,United States,0,0,0,0,"['alreadyover', 'Hyperinflation', 'bubble', 'B...",NaN,"[User(username='GOP_CAPITAL', id=1134254394394...",en,NaN,NaN
3,1642315840816021504,2023-04-01 23:59:43+00:00,a new block was found on the network were at ...,1448763322812276737,satsleft,satsleft.info,2021-10-14 21:31:42+00:00,594,False,mempool,0,0,0,0,"['Bitcoin', 'bitcoin', 'sats']",NaN,NaN,en,NaN,NaN
4,1642315775867334656,2023-04-01 23:59:28+00:00,bitcoin price index,2395138046,WorldCoinIndex,WorldCoinIndex,2014-03-17 21:39:08+00:00,12768,False,NaN,0,0,0,0,"['USD', 'EUR', 'CNY', 'GBP', 'RUB']",NaN,NaN,en,NaN,NaN
5,1642315771589308416,2023-04-01 23:59:27+00:00,"i just listed inscription for 1 btc ,48575 on ...",36308803,t1n0_29,tino29.btc,2009-04-29 07:05:13+00:00,1263,False,Area 51,6,0,0,0,NaN,NaN,"[User(username='ordinalswallet', id=1620686423...",en,NaN,NaN
6,1642315764894953472,2023-04-01 23:59:25+00:00,dont miss out on the next big thing in alpha t...,1391067160198455302,yashi0_8,Alpha Trade,2021-05-08 16:27:23+00:00,1681,False,Eternally on the blockchain.,2,0,0,0,"['crypto', 'SHIB', 'ALPHA', 'Bitcoin', 'Ethere...",NaN,NaN,en,NaN,NaN
7,1642315760881238021,2023-04-01 23:59:24+00:00,"government doesnt make its own money, so they...",914674650780598272,porzo8,Porzo8/OGIE,2017-10-02 02:13:25+00:00,311,False,"Colorado, USA",0,0,0,0,NaN,NaN,"[User(username='zerohedge', id=18856867, displ...",en,NaN,NaN
8,1642315751238369281,2023-04-01 23:59:22+00:00,have faith even when you feel alone in that fa...,1229269844652109824,rsinvest9,rsinvest,2020-02-17 05:02:35+00:00,249,False,unusual whales,1,0,0,0,['bitcoin'],NaN,NaN,en,NaN,NaN
9,1642315748751167488,2023-04-01 23:59:21+00:00,i have not sold any bitcoin since 2020,309272893,AndresEp94,Andres Espinosa,2011-06-01 20:33:42+00:00,89,False,Guadalajara,2,0,0,0,NaN,NaN,"[User(username='BitcoinMagazine', id=361289499...",en,NaN,NaN
